In [1]:
import os, time, re, json, subprocess, sys
import pandas as pd
import importlib.util as il

if None in [il.find_spec('python-ulid'), il.find_spec('pyperclip')]:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'python-ulid']);
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'pyperclip']);
    
from ulid import ULID
import pyperclip

def gen_ulid():
    return str(ULID.from_timestamp(time.time()))

def convert_coord(c):
    c = str(c)
    j = len(c) - 6
    d = int(c[0:2 + j])
    m = int(c[2 + j:4 + j])
    s = float(c[4 + j:6 + j] + '.' + c[6 + j:])
    q = 1 if j == 0 else -1
    coord = round(q * (d + m / 60 + s / 3600), 6)
    
    return coord

def pprint(dict):
    print(json.dumps(dict, indent=2))

def comma_followed_by_number(s):
    for i, char in enumerate(s[:-1]):
        if char == ',' and s[i+1].isdigit():
            return True
    return False

def extract_table_section_from_file(section_header, filename, offset=0):
    offset *= 3
    section_header = '******* ' + section_header + ' *******'

    downloads_folder = os.path.join(os.path.expanduser("~"), "Downloads")
    with open(os.path.join(downloads_folder, filename), "r") as file:
        lines = file.readlines()

    extracted_lines = []
    inside_section = False
    end_marker_count = 0

    for line in lines:
        if section_header in line:
            inside_section = True
            extracted_lines.append(line)
            continue

        if inside_section:
            if end_marker_count > offset:
                extracted_lines.append(line)
            # Count lines that are mostly dashes
            if line.strip().startswith('---'):
                end_marker_count += 1
                if end_marker_count >= 3 + offset:
                    break

    return "".join(extracted_lines)

def remove_dash_lines(text):
    cleaned_lines = [
        line for line in text.splitlines()
        if not line.strip().startswith("---")
    ]
    return "\n".join(cleaned_lines)

def convert_pipe_text_to_csv(multi_line_text):
    csv_lines = []
    for line in multi_line_text.splitlines():
        if not line.strip():
            continue
        if '|' not in line:
            continue
        
        fields = [field.strip() for field in line.strip('|').split('|')]
        csv_line = '|'.join(fields)
        csv_lines.append(csv_line)

    return '\n'.join(csv_lines)

def csv_text_to_dataframe(csv_text):
    lines = [line.strip() for line in csv_text.strip().split('\n') if line.strip()]
    
    headers = [h.strip() for h in lines[0].split('|')]
    
    data = []
    for line in lines[1:]:
        fields = [f.strip() for f in line.split('|')]
        data.append(fields)
    
    df = pd.DataFrame(data, columns=headers)
    return df

def read_adaptation_section(section_header, filename, offset=0):
    text = extract_table_section_from_file(section_header, filename, offset)
    text = remove_dash_lines(text)
    text = convert_pipe_text_to_csv(text)
    
    return csv_text_to_dataframe(text)

In [28]:
downloads_path = os.path.join(os.path.expanduser("~"), "Downloads")
cols = ["Arts Id", "Route Type", "Route Id", "Owning Facility", "Route Fix",
    "ARTS Fix", "Unique Name", "Altitude Lower", "Altitude Upper", "Ac Class Criterias"]
df = pd.read_csv(os.path.join(downloads_path, "route_based.csv"), usecols=cols)

facility = df[df['Arts Id'] == 'FFF']

s = []
dp_df = facility[(facility['Route Type'] == 'DP') | (facility['Route Type'] == 'AIRWAY')]
for index, row in dp_df.iterrows():
    if row['Route Type'] == 'DP':
        route_id = re.sub(r'\d+$', '#', row['Route Id'])
    else:
        route_id = row['Route Id']
    data = route_id + (' ' + str(row['Route Fix'])).replace(' nan', '')
    pattern = row['ARTS Fix']
    
    if not pd.isna(row['Altitude Lower']):
        aoa = int(row['Altitude Lower'] / 100)
        data += ' AOA/' + f"{aoa:03d}"
    if not pd.isna(row['Altitude Upper']):
        aob = int(row['Altitude Upper'] / 100)
        data += ' AOB/' + f"{aob:03d}"
    if not pd.isna(row['Ac Class Criterias']):
        acc = row['Ac Class Criterias']
        if not('NATJ' in acc or 'NATM' in acc or \
               'ZMAQ' in acc or 'ZMAP' in acc):
            continue
        
        data += ' TYP/'
        if 'NATJ' in acc:
            data += 'J'
        if 'NATM' in acc:
            data += 'T'
        if 'ZMAQ' in acc or 'ZMAP' in acc:
            data += 'P'

    out = 'fix_pattern[\'' + data + '\'] = \'' + pattern + '\'\n'
    if not out in s:
        s.append(out)

adr_df = facility[facility['Route Type'] == 'ADR']
for index, row in adr_df.iterrows():
    if pd.isna(row['Route Fix']):
        continue
    elif row['Route Fix'][3:] == 'WX':
        continue
    
    data = row['Route Fix']
    pattern = row['ARTS Fix']
    
    if not pd.isna(row['Altitude Lower']):
        aoa = int(row['Altitude Lower'] / 100)
        data += ' AOA/' + f"{aoa:03d}"
    if not pd.isna(row['Altitude Upper']):
        aob = int(row['Altitude Upper'] / 100)
        data += ' AOB/' + f"{aob:03d}"
    if not pd.isna(row['Ac Class Criterias']):
        acc = row['Ac Class Criterias']
        if not('NATJ' in acc or 'NATM' in acc or \
               'ZMAQ' in acc or 'ZMAP' in acc):
            continue
        
        data += ' TYP/'
        if 'NATJ' in acc:
            data += 'J'
        if 'NATM' in acc:
            data += 'T'
        if 'ZMAQ' in acc or 'ZMAP' in acc:
            data += 'P'

    out = 'fix_pattern[\'' + data + '\'] = \'' + pattern + '\'\n'
    if not out in s:
        s.append(out)

s_out = ''.join(sorted(s, key=lambda x: x.split("=")[1]))
pyperclip.copy(s_out)
print(s_out)

fix_pattern['AABER'] = 'AAB'
fix_pattern['V599 THNDR AOA/051 AOB/999'] = 'BOD'
fix_pattern['CHARO'] = 'CHR'
fix_pattern['CSHEL'] = 'CPP'
fix_pattern['PULEC'] = 'CPP'
fix_pattern['CSHEL# CSHEL'] = 'CSH'
fix_pattern['CSHEL'] = 'CSH'
fix_pattern['V599 THNDR AOA/000 AOB/051'] = 'DBO'
fix_pattern['DYLYN'] = 'DYL'
fix_pattern['DYLYN'] = 'ELA'
fix_pattern['MOOKY'] = 'GUL'
fix_pattern['PIKKR AOA/000 AOB/101'] = 'GUL'
fix_pattern['PIKKR AOA/101 AOB/990'] = 'GUL'
fix_pattern['PIKKR AOA/000 AOB/100'] = 'GUL'
fix_pattern['PIKKR AOA/101 AOB/999'] = 'GUL'
fix_pattern['MOOKY AOA/000 AOB/100'] = 'GUL'
fix_pattern['MOOKY AOA/101 AOB/999'] = 'GUL'
fix_pattern['IMOCK'] = 'IMO'
fix_pattern['V7 ROGAN AOA/000 AOB/100'] = 'LLL'
fix_pattern['RIGOR AOA/101 AOB/999'] = 'MAR'
fix_pattern['RIGOR AOA/000 AOB/101'] = 'MAR'
fix_pattern['KARTR AOA/000 AOB/101'] = 'MAR'
fix_pattern['KARTR AOA/101 AOB/990'] = 'MAR'
fix_pattern['ROGAN AOA/100 AOB/999'] = 'R0G'
fix_pattern['ROGAN'] = 'ROG'
fix_pattern['ROGAN AOA/091 AOB/

In [31]:
def sort_fix_pattern(k):
    if '#' in k.split(' ')[0]:
        priority = 0
    elif any(c.isdigit() for c in k.split(' ')[0]):
        priority = 1
    else:
        priority = 2
    return (priority, k)

fix_pattern = dict(sorted(fix_pattern.items(), key=lambda item: sort_fix_pattern(item[0])))
# pprint(fix_pattern)

scratchpads = []
for s in fix_pattern:
    p = {}
    p['id'] = gen_ulid()

    sp = s
    if ' AOA/' in s:
        aoa = int(s.split(' AOA/')[1][0:3])
        if aoa != 0:
            p['minAltitude'] = aoa
        sp = re.sub(r' AOA/\d{3}', '', sp)
    if ' AOB/' in s:
        aob = int(s.split(' AOB/')[1][0:3])
        if aob <= 999:
            p['maxAltitude'] = aob
        sp = re.sub(r' AOB/\d{3}', '', sp)

    if ' DEP/' in s:
        p['airportIds'] = s.split(' DEP/')[1].split(' ')[0].split('/')
        sp = re.sub(r' DEP(/\w{1,})+', '', sp)
    
    p['searchPattern'] = sp
    p['template'] = fix_pattern[s]
    scratchpads.append(p)
pyperclip.copy(scratchpads)